<a href="https://colab.research.google.com/github/jon-nowacki/Optimization-Models/blob/main/Ford_Fulkerson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ford-Fulkerson

This code demonstrates the Ford-Fulkerson algorithm using Pyomo to solve a flow network problem. It creates a Pyomo model, defines decision variables, an objective function, capacity constraints, and flow conservation constraints to find the maximum flow from a specified source node to a sink node in a given flow network represented by capacities. Adjust the capacities and source/sink nodes as needed for your specific network.

In [2]:
!pip install -q pyomo > /dev/null
!apt-get install -y -qq glpk-utils > /dev/null

In [5]:
from pyomo.environ import *

# Create a function to run the Ford-Fulkerson algorithm
def ford_fulkerson(capacity, source, sink):
    # Create the Pyomo model
    model = ConcreteModel()

    # Set of nodes
    nodes = sorted(list(set([i for i, j in capacity.keys()] + [j for i, j in capacity.keys()])))

    # Define the decision variables: flow between nodes
    model.flow = Var(nodes, nodes, within=NonNegativeReals)

    # Define the objective function: maximize flow to the sink
    def obj_rule(model):
        # Maximize the flow entering the sink and minimize flow leaving the sink
        return sum(model.flow[i, sink] for i in nodes if (i, sink) in capacity) - sum(model.flow[sink, j] for j in nodes if (sink, j) in capacity)
    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Define the constraints (capacity constraints and flow conservation)
    def capacity_constraints_rule(model, i, j):
        # Capacity constraints: flow on each edge must be within the specified capacity
        return (0, model.flow[i, j], capacity.get((i, j), 0))
    model.capacity_constraints = Constraint(capacity.keys(), rule=capacity_constraints_rule)

    def flow_conservation_rule(model, node):
        # Flow conservation at each node except source and sink
        if node != source and node != sink:
            # Flow into the node must equal flow out of the node
            return sum(model.flow[i, node] for i in nodes if (i, node) in capacity) - sum(model.flow[node, j] for j in nodes if (node, j) in capacity) == 0
        return Constraint.Skip  # Skip constraints for the source and sink nodes
    model.flow_conservation = Constraint(nodes, rule=flow_conservation_rule)

    # Solve the model using the 'glpk' solver
    solver = SolverFactory('glpk')
    solver.solve(model)

    # Return the maximum flow
    return model.obj()

# Example flow network capacities
capacities = {
    (1, 2): 3,  # Edge (1, 2) with capacity 3
    (1, 3): 2,  # Edge (1, 3) with capacity 2
    (2, 3): 2,  # Edge (2, 3) with capacity 2
    (2, 4): 2,  # Edge (2, 4) with capacity 2
    (3, 4): 3   # Edge (3, 4) with capacity 3
}

# Set the source and sink nodes
source_node = 1
sink_node = 4

# Find the maximum flow
max_flow = ford_fulkerson(capacities, source_node, sink_node)
print(f"The maximum flow from node {source_node} to node {sink_node} is: {max_flow}")


The maximum flow from node 1 to node 4 is: 5.0
